In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=3

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=3


In [2]:
import os
import pymatgen
import matplotlib.pyplot as plt
import pandas as pd
from tqdm.notebook import trange, tqdm
from data import get_dichalcogenides_innopolis_202105

In [3]:
structures = get_dichalcogenides_innopolis_202105()

  0%|          | 0/3480 [00:00<?, ?it/s]

/temporary/pymatgen/pymatgen/io/cif.py:1123: UserWarning:

Issues encountered while parsing CIF: Some fractional co-ordinates rounded to ideal values to avoid issues with finite precision.



In [4]:
import matplotlib

from megnet.models import MEGNetModel
from megnet.data.graph import GaussianDistance
from megnet.data.crystal import CrystalGraph
from megnet.data.molecule import MolecularGraph
from megnet.utils.preprocessing import StandardScaler
from megnet.callbacks import ModelCheckpointMAE
from pymatgen.core import Lattice, Structure, Molecule

import tensorflow as tf
import numpy as np

In [5]:
import wandb
from wandb.keras import WandbCallback

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
train, test = train_test_split(structures, test_size=0.25, random_state=42)

In [8]:
wandb.init(project='ai4material_design', entity='kazeev')

wandb: Currently logged in as: kazeev (use `wandb login --relogin` to force relogin)


In [9]:
config = wandb.config
config.target = "energy_per_atom"

In [10]:
gc = CrystalGraph(bond_converter=GaussianDistance(np.linspace(0, 5, 100), 0.5), cutoff=6)
model = MEGNetModel(nfeat_edge=100, nfeat_global=2, graph_converter=gc, npass=5)

In [11]:
scaler = StandardScaler.from_training_data(train["initial_structure"],
                                           train[config.target], is_intensive=True)
model.target_scaler = scaler

In [13]:
model.train(train["initial_structure"], train[config.target],
            validation_structures=test["initial_structure"],
            validation_targets=test[config.target],
            callbacks=[WandbCallback()],
            epochs=100, verbose=1, patience=1000, prev_model="callback/val_mae_00085_0.034367.hdf5")

Epoch 1/100
21/21 [==============================] - 86s 4s/step - loss: 0.0480


INFO:megnet.callbacks:
Epoch 00001: val_mae improved from inf to 0.03619, saving model to callback/val_mae_00001_0.036195.hdf5


Epoch 2/100
21/21 [==============================] - 86s 4s/step - loss: 0.0492
Epoch 3/100
21/21 [==============================] - 86s 4s/step - loss: 0.0492
Epoch 4/100
21/21 [==============================] - 86s 4s/step - loss: 0.0498
Epoch 5/100
21/21 [==============================] - 86s 4s/step - loss: 0.0498
Epoch 6/100
21/21 [==============================] - 86s 4s/step - loss: 0.0490


INFO:megnet.callbacks:
Epoch 00006: val_mae improved from 0.03619 to 0.03404, saving model to callback/val_mae_00006_0.034044.hdf5


Epoch 7/100
21/21 [==============================] - 87s 4s/step - loss: 0.0486
Epoch 8/100
21/21 [==============================] - 87s 4s/step - loss: 0.0535
Epoch 9/100
21/21 [==============================] - 86s 4s/step - loss: 0.0482
Epoch 10/100
21/21 [==============================] - 86s 4s/step - loss: 0.0540
Epoch 11/100
21/21 [==============================] - 86s 4s/step - loss: 0.0541
Epoch 12/100
21/21 [==============================] - 86s 4s/step - loss: 0.0526
Epoch 13/100
21/21 [==============================] - 86s 4s/step - loss: 0.0485
Epoch 14/100
21/21 [==============================] - 86s 4s/step - loss: 0.0490
Epoch 15/100
21/21 [==============================] - 86s 4s/step - loss: 0.0484
Epoch 16/100
21/21 [==============================] - 86s 4s/step - loss: 0.0480
Epoch 17/100
21/21 [==============================] - 86s 4s/step - loss: 0.0483
Epoch 18/100
21/21 [==============================] - 85s 4s/step - loss: 0.0495
Epoch 19/100
21/21 [===========

INFO:megnet.callbacks:
Epoch 00023: val_mae improved from 0.03404 to 0.03351, saving model to callback/val_mae_00023_0.033510.hdf5


Epoch 24/100
21/21 [==============================] - 87s 4s/step - loss: 0.0486
Epoch 25/100
21/21 [==============================] - 86s 4s/step - loss: 0.0491
Epoch 26/100
21/21 [==============================] - 92s 4s/step - loss: 0.0491
Epoch 27/100
21/21 [==============================] - 92s 4s/step - loss: 0.0488
Epoch 28/100
21/21 [==============================] - 89s 4s/step - loss: 0.0538
Epoch 29/100
21/21 [==============================] - 89s 4s/step - loss: 0.0539
Epoch 30/100
21/21 [==============================] - 94s 4s/step - loss: 0.0483
Epoch 31/100
21/21 [==============================] - 112s 5s/step - loss: 0.0480
Epoch 32/100
21/21 [==============================] - 121s 6s/step - loss: 0.0491
Epoch 33/100
21/21 [==============================] - 183s 8s/step - loss: 0.0500
Epoch 34/100
21/21 [==============================] - 148s 7s/step - loss: 0.0497
Epoch 35/100
21/21 [==============================] - 119s 6s/step - loss: 0.0502
Epoch 36/100
21/21 [===